In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd

from data_parameters import DataParamMode, DataParamOptim
from data_parameters import DataParameterManager

# 1D dataset training: Disturb all label values of specific instances

In [4]:
CLASSES = ['background','ventricle', 'myocard', 'vein']

class MiniDataset(Dataset):
    # A dataset containing the same images and only background labels.
    def __init__(self):
        self.len = 10
        self.images = -2*torch.ones((self.len, 35))
        self.labels = torch.stack(
            [
                torch.ones((self.len)), # background
                torch.zeros((self.len)), # venctricle
                torch.zeros((self.len)), # myocard
                torch.zeros((self.len)), # vein
            ],
            dim=-1
        ).long()
        # self.labels[0,0] = 1
        self.disturbed_idxs = []

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        if idx in self.disturbed_idxs:
            # Disturb instance here: Increase background label value
            label = self.labels[idx]*3
        else:
            label = self.labels[idx]

        return {'d_idx': idx, 'image': self.images[idx], 'label': label}

    def set_disturbed_idxs(self, idxs):
        self.disturbed_idxs = idxs

        

class MiniNet(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.layer = torch.nn.AdaptiveAvgPool1d(1)
        self.linear = torch.nn.Linear(1, len(CLASSES), bias=False)

    def forward(self, _input):
        _output = self.layer(_input.unsqueeze(1)).squeeze(1)
        _output = self.linear(_output)**2
        return _output

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

config = dotdict({
    # Data parameter config
    'data_param_mode': DataParamMode.ONLY_INSTANCE_PARAMS,
    'init_class_param': 1.0, 
    'lr_class_param': 0.1,
    'init_inst_param': 1.0, 
    'lr_inst_param': 0.1,
    'wd_inst_param': 0.0,
    'wd_class_param': 0.0,
    
    'skip_clamp_data_param': False,
    'clamp_sigma_min': np.log(1/20),
    'clamp_sigma_max': np.log(20),
    'optim_algorithm': DataParamOptim.ADAM,
    'optim_options': dict(
        momentum=.9
        # betas=(0.9, 0.999)
    )
})

torch.manual_seed(0)
# random.seed(0)
np.random.seed(0)

data = MiniDataset()

train_dataloader = DataLoader(data, 3, shuffle=True)
# Problem: SGD, w/o momentum, when disturbed parameters are in unbalanced batches (varying number of disturbed parameters in minibatch)
# independent of param group definition

criterion = torch.nn.MSELoss()

epochs = 100
data.set_disturbed_idxs([0,1,7])
print(f"Disturbed instances are: {data.disturbed_idxs}")

loss_comparison = {}

for mode in DataParamMode:
    print(f"########### Testing mode={mode}")
    if mode in [DataParamMode.COMBINED_INSTANCE_CLASS_PARAMS, DataParamMode.SEPARATE_INSTANCE_CLASS_PARAMS]:
        config.init_class_param = 1.0
        config.init_inst_param = 0.001
    else:
        config.init_class_param = 1.0
        config.init_inst_param = 1.0

    for optim_algorithm in DataParamOptim:
        if optim_algorithm == DataParamOptim.ADAM:
            config.optim_options = dict()
        else:
            config.optim_options = dict(momentum=.9)
        
        print(f"##### Using {optim_algorithm}")
        config.optim_algorithm = optim_algorithm
        config.data_param_mode = mode

        # Reset determinism
        torch.manual_seed(0)
        # random.seed(0)
        np.random.seed(0)

        # Setup data parameter manager
        dpm = DataParameterManager(instance_keys=range(len(data)), class_keys=CLASSES, config=config, device='cpu')

        # Setup network
        net = MiniNet()
        
        optimizer = torch.optim.Adam(net.parameters(), lr=.01)
        # data.set_disturbed_idxs([0,1])
        for epx in range(epochs):
            # print("epoch", epx)
            for b_idx, sample in enumerate(train_dataloader):
                # if fail_var():
                    # raise(ValueError(f"err at {epx} {b_idx}"))
                # print("batch", b_idx)
                image, label = sample['image'], sample['label']

                logits = net(image)
                _, loss = dpm.do_basic_train_step(
                    criterion, 
                    logits, 
                    label, 
                    optimizer, 
                    inst_keys=sample['d_idx'].tolist(),
                    scaler=None)
        
        print(f"Loss={loss}")
        mode_dict = loss_comparison.get(mode, {})
        mode_dict[optim_algorithm] = loss
        loss_comparison[mode] = mode_dict

        with torch.no_grad():
            df = pd.DataFrame(dpm.get_data_parameters_dict())
            for ridx, row in df.iterrows():
                for cidx, elem in row.iteritems():
                    df[cidx][ridx] = np.array(elem).item()#np.exp(elem).item()
        pd.options.display.float_format = '{:.2f}'.format
        display(df)
        print()
        print()

print("########### Loss comparison:")
pd.options.display.float_format = '{:.6f}'.format
display(pd.DataFrame(loss_comparison))

Disturbed instances are: [0, 1, 7]
########### Testing mode=DataParamMode.ONLY_INSTANCE_PARAMS
##### Using DataParamOptim.ADAM
Initialized instance data parameters with: 1.0
Loss=0.00030233757570385933


,0,1,2,3,4,5,6,7,8,9
0,0.44,0.44,1.54,1.54,1.54,1.54,1.54,0.44,1.54,1.54




##### Using DataParamOptim.SGD
Initialized instance data parameters with: 1.0
Loss=0.0006040844018571079


,0,1,2,3,4,5,6,7,8,9
0,0.52,0.52,1.62,1.62,1.62,1.63,1.62,0.52,1.63,1.62




##### Using DataParamOptim.SPARSE_SGD
Initialized instance data parameters with: 1.0
Loss=0.0006040844018571079


...

""




##### Using DataParamOptim.SPARSE_SGD
Loss=0.5360684990882874


""




########### Loss comparison:


,DataParamMode.ONLY_INSTANCE_PARAMS,DataParamMode.ONLY_CLASS_PARAMS,DataParamMode.COMBINED_INSTANCE_CLASS_PARAMS,DataParamMode.SEPARATE_INSTANCE_CLASS_PARAMS,DataParamMode.DISABLED
DataParamOptim.ADAM,0.000302,0.438544,0.000204,0.000122,0.536068
DataParamOptim.SGD,0.000604,0.592760,0.000206,0.000189,0.536068
DataParamOptim.SPARSE_SGD,0.000604,0.592760,0.000206,0.000189,0.536068


# 2D training: Disturb only one label class of all instances. This should "activate" class parameters

In [6]:
CLASSES = ['background','ventricle','myocard', 'aorta', 'splenic_vein']
SPATIAL_DIM = 4

class MiniDataset2D(Dataset):
    # 9 images are filled with 3x1.0, 3x2.0, 3x3.0
    # 9 labels carry the corresponding one-hot-class 3x1, 3x2, 3x3
    def __init__(self):
        self.len = 9
        # Images have 9x1x4x4 shape (onehot of 4x4 segmentation)
        self.images = torch.zeros((self.len, 1, SPATIAL_DIM,SPATIAL_DIM))
        self.images[:self.len//3] = 1*torch.ones((self.len//3, 1, SPATIAL_DIM,SPATIAL_DIM))
        self.images[self.len//3:2*self.len//3] = 2*torch.ones((2*self.len//3-self.len//3, 1, SPATIAL_DIM,SPATIAL_DIM))
        self.images[2*self.len//3:] = 3*torch.ones((self.len-2*self.len//3, 1, SPATIAL_DIM,SPATIAL_DIM))

        # Labels have 9x4x4x3 shape (onehot of 4x4 segmentation)
        self.labels = torch.zeros((self.len, SPATIAL_DIM,SPATIAL_DIM))
        self.labels[:self.len//3] = 1*torch.ones((self.len//3, SPATIAL_DIM,SPATIAL_DIM))
        self.labels[self.len//3:2*self.len//3] = 2*torch.ones((2*self.len//3-self.len//3, SPATIAL_DIM,SPATIAL_DIM))
        self.labels[2*self.len//3:] = 3*torch.ones((self.len-2*self.len//3, SPATIAL_DIM,SPATIAL_DIM))
        self.labels = torch.nn.functional.one_hot(self.labels.long(), num_classes=len(CLASSES))
        self.disturbed_idxs = []

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        if idx in self.disturbed_idxs:
            label = self.labels[idx].roll(1, dims=-1) # Roll onehot dimension of label to disturb labels
            # Disturb instances here:
            label[idx] = 0
        else:
            label = self.labels[idx]

        return {'d_idx': idx, 'image': self.images[idx], 'label': label}

    def set_disturbed_idxs(self, idxs):
        self.disturbed_idxs = idxs



class MiniNet2D(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.conv = torch.nn.Conv2d(1, len(CLASSES), (1,1), bias=False)

    def forward(self, _input):
        _output = self.conv(_input)**2
        return _output.permute(0,2,3,1)


class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

config = dotdict({
    # Data parameter config
    'data_param_mode': DataParamMode.ONLY_INSTANCE_PARAMS,
    'init_class_param': 1.00, 
    'lr_class_param': 0.1,
    'init_inst_param': 0.001, 
    'lr_inst_param': 0.1,
    'wd_inst_param': 0.0,
    'wd_class_param': 0.0,
    
    'skip_clamp_data_param': False,
    'clamp_sigma_min': np.log(1/20),
    'clamp_sigma_max': np.log(20),
    'optim_algorithm': DataParamOptim.ADAM,
    'optim_options': dict(
        momentum=.9
        # betas=(0.9, 0.999)
    )
})

torch.manual_seed(0)
# random.seed(0)
np.random.seed(0)

data2D = MiniDataset2D()

train_dataloader = DataLoader(data2D, 3, shuffle=True)
# Problem: SGD, w/o momentum, when disturbed parameters are in unbalanced batches (varying number of disturbed parameters in minibatch)
# independent of param group definition

criterion = torch.nn.MSELoss()

epochs = 100
data.set_disturbed_idxs([0,1,7])
print(f"Disturbed instances are: {data.disturbed_idxs}")

loss_comparison = {}

for mode in DataParamMode:
    print(f"########### Testing mode={mode}")
    if mode in [DataParamMode.COMBINED_INSTANCE_CLASS_PARAMS, DataParamMode.SEPARATE_INSTANCE_CLASS_PARAMS]:
        config.init_class_param = 1.0
        config.init_inst_param = 0.001
    else:
        config.init_class_param = 1.0
        config.init_inst_param = 1.0

    for optim_algorithm in DataParamOptim:
        if optim_algorithm == DataParamOptim.ADAM:
            config.optim_options = dict()
        else:
            config.optim_options = dict(momentum=.9)
        
        print(f"##### Using {optim_algorithm}")
        config.optim_algorithm = optim_algorithm
        config.data_param_mode = mode

        # Reset determinism
        torch.manual_seed(0)
        # random.seed(0)
        np.random.seed(0)

        # Setup data parameter manager
        dpm = DataParameterManager(instance_keys=range(len(data)), class_keys=CLASSES, config=config, device='cpu')

        # Setup network
        net = MiniNet2D()
        
        optimizer = torch.optim.Adam(net.parameters(), lr=.01)
        # data.set_disturbed_idxs([0,1])
        for epx in range(epochs):
            # print("epoch", epx)
            for b_idx, sample in enumerate(train_dataloader):
                # if fail_var():
                    # raise(ValueError(f"err at {epx} {b_idx}"))
                # print("batch", b_idx)
                image, label = sample['image'], sample['label']

                logits = net(image)
                _, loss = dpm.do_basic_train_step(
                    criterion, 
                    logits, 
                    label, 
                    optimizer, 
                    inst_keys=sample['d_idx'].tolist(),
                    scaler=None)
        
        print(f"Loss={loss}")
        mode_dict = loss_comparison.get(mode, {})
        mode_dict[optim_algorithm] = loss
        loss_comparison[mode] = mode_dict

        with torch.no_grad():
            df = pd.DataFrame(dpm.get_data_parameters_dict())
            for ridx, row in df.iterrows():
                for cidx, elem in row.iteritems():
                    df[cidx][ridx] = np.array(elem).item()#np.exp(elem).item()
        pd.options.display.float_format = '{:.2f}'.format
        display(df)
        print()
        print()

print("########### Loss comparison:")
pd.options.display.float_format = '{:.6f}'.format
display(pd.DataFrame(loss_comparison))

Disturbed instances are: [0, 1, 7]
########### Testing mode=DataParamMode.ONLY_INSTANCE_PARAMS
##### Using DataParamOptim.ADAM
Initialized instance data parameters with: 1.0
Loss=0.10917415469884872


,0,1,2,3,4,5,6,7,8,9
0,0.89,0.81,0.81,1.26,1.26,1.26,2.12,2.12,2.12,1.00




##### Using DataParamOptim.SGD
Initialized instance data parameters with: 1.0
Loss=0.13058529794216156


,0,1,2,3,4,5,6,7,8,9
0,0.93,0.93,0.93,1.09,1.09,1.09,2.89,2.89,2.89,1.00




##### Using DataParamOptim.SPARSE_SGD
Initialized instance data parameters with: 1.0
Loss=0.13058529794216156


...

""




##### Using DataParamOptim.SPARSE_SGD
Loss=0.08633123338222504


""




########### Loss comparison:


,DataParamMode.ONLY_INSTANCE_PARAMS,DataParamMode.ONLY_CLASS_PARAMS,DataParamMode.COMBINED_INSTANCE_CLASS_PARAMS,DataParamMode.SEPARATE_INSTANCE_CLASS_PARAMS,DataParamMode.DISABLED
DataParamOptim.ADAM,0.109174,0.005861,0.006309,0.012520,0.086331
DataParamOptim.SGD,0.130585,0.031545,0.015610,0.026801,0.086331
DataParamOptim.SPARSE_SGD,0.130585,0.031545,0.015610,0.026801,0.086331
